## Time travel

In a typical chatbot workflow, the user interacts with the bot one or more times to accomplish a task. Memory and a human-in-the-loop enable checkpoints in the graph state and control future responses.
What if you want a user to be able to start from a previous response and explore a different outcome? Or what if you want users to be able to rewind your chatbot's work to fix mistakes or try a different strategy, something that is common in applications like autonomous software engineers?

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load from .env file

api_key = os.getenv("TAVILY_API_KEY")

print("API key loaded:", api_key[:4] + "..." if api_key else "Not found")

API key loaded: tvly...


In [2]:
from langchain.chat_models import init_chat_model # type: ignore

# Follow the steps here to configure your credentials:
# https://docs.aws.amazon.com/bedrock/latest/userguide/getting-started.html

llm = init_chat_model(
    "anthropic.claude-3-haiku-20240307-v1:0",
    model_provider="bedrock_converse",
)

#### 1- Rewind your graph
Rewind your graph by fetching a checkpoint using the graph's get_state_history method. You can then resume execution at this previous point in time.

In [3]:
from typing import Annotated

from langchain_tavily import TavilySearch
from langchain_core.messages import BaseMessage
from typing_extensions import TypedDict

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearch(max_results=2)
tools = [tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

#### 2- Add steps
Add steps to your graph. Every step will be checkpointed in its state history:

In [4]:
config = {"configurable": {"thread_id": "1"}}
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "I'm learning LangGraph. "
                    "Could you do some research on it for me?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================

[{'type': 'tool_use', 'name': 'tavily_search', 'input': {'query': 'LangGraph', 'search_depth': 'advanced'}, 'id': 'tooluse_RC3JTJIFTXCAUkwA-XS6bA'}]
Tool Calls:
  tavily_search (tooluse_RC3JTJIFTXCAUkwA-XS6bA)
 Call ID: tooluse_RC3JTJIFTXCAUkwA-XS6bA
  Args:
    query: LangGraph
    search_depth: advanced
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.ibm.com/think/topics/langgraph", "title": "What is LangGraph? - IBM", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. It provides a set o

In [5]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!. could you do some research on building autonomous agents with LangGraph?"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!. could you do some research on building autonomous agents with LangGraph?
================================== Ai Message ==================================

[{'type': 'text', 'text': "Okay, let's dive deeper into how to build autonomous agents using LangGraph."}, {'type': 'tool_use', 'name': 'tavily_search', 'input': {'search_depth': 'advanced', 'query': 'building autonomous agents with langgraph'}, 'id': 'tooluse_SyGOMYuCRauGHRG_nG8xlw'}]
Tool Calls:
  tavily_search (tooluse_SyGOMYuCRauGHRG_nG8xlw)
 Call ID: tooluse_SyGOMYuCRauGHRG_nG8xlw
  Args:
    search_depth: advanced
    query: building autonomous agents with langgraph
================================= Tool Message =================================
Name: tavily_search

{"query": "building autonomous agents with langgraph", "follow_up_questions": null, "answer": null, "images": [], "res

#### 3- Replay the fullstate history

In [6]:
to_replay = None
for state in graph.get_state_history(config):
    print("Num Messages: ", len(state.values["messages"]), "Next: ", state.next)
    print("-" * 80)
    if len(state.values["messages"]) == 6:
        # We are somewhat arbitrarily selecting a specific state based on the number of chat messages in the state.
        to_replay = state

Num Messages:  8 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  7 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  6 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
-----------------------------------------

Checkpoints are saved for every step of the graph. This spans invocations so you can rewind across a full thread's history.

#### Resume from a checkpoint
Resume from the to_replay state, which is after the chatbot node in the second graph invocation. Resuming from this point will call the action node next.

In [ ]:
# Print the state we want to replay. Thsi will be 6 in above
print(to_replay.next)
print(to_replay.config)

('tools',)
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f03b4d9-8f76-6b86-8006-44d12b75bc21'}}


#### 4- Load a state from a moment-in-time
The checkpoint's to_replay.config contains a checkpoint_id timestamp. Providing this checkpoint_id value tells LangGraph's checkpointer to load the state from that moment in time.

In [8]:
# The `checkpoint_id` in the `to_replay.config` corresponds to a state we've persisted to our checkpointer.
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================== Ai Message ==================================

[{'type': 'text', 'text': "Okay, let's dive deeper into how to build autonomous agents using LangGraph."}, {'type': 'tool_use', 'name': 'tavily_search', 'input': {'search_depth': 'advanced', 'query': 'building autonomous agents with langgraph'}, 'id': 'tooluse_SyGOMYuCRauGHRG_nG8xlw'}]
Tool Calls:
  tavily_search (tooluse_SyGOMYuCRauGHRG_nG8xlw)
 Call ID: tooluse_SyGOMYuCRauGHRG_nG8xlw
  Args:
    search_depth: advanced
    query: building autonomous agents with langgraph
================================= Tool Message =================================
Name: tavily_search

{"query": "building autonomous agents with langgraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.coursera.org/learn/packt-building-autonomous-ai-agents-with-langgraph-oyjym", "title": "Building Autonomous AI Agents with LangGraph | Coursera", "content": "What you'll learn. Build autonomou